In [24]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =10

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

# MODEL CONFIG
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
os.chdir("data")

In [3]:
df = pd.read_csv("CleanedData.csv")

In [4]:
df["Category_name"].value_counts()

tv, television, led                2266
drive, storage, hard               2105
headphone, earbud, bluetooth       1801
tv, 4k, television                 1636
headphone                          1580
receiver, amplifier, home          1541
car, gps, dash                     1464
camera, shoot, point               1386
tablet, computer, ipad             1363
camera, mirrorless                 1302
laptop, computer                   1210
speaker, portable, bluetooth       1202
player, ipod                       1098
sound, speaker, home               1088
car, speaker, subwoofer            1072
speaker, audio                     1035
case, computer                      926
laptop, computer, macbook           925
speaker, bluetooth, mobile          841
phone, cell                         789
mount, tv, stand                    783
camera, camcorder, action           750
wireless, speaker, audio            696
lens, camera                        683
monitor, computer                   654


## Price Elasticity for "Speakers"

In [5]:
dfx = df[df["Category_name"].str.contains("speaker",case=False)]
dfx= dfx[dfx["Year"]==2017]

In [6]:
dfx = dfx.reset_index(drop=True)

In [7]:
dfx.sort_values(by=['Date'],ignore_index=True)

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Date
0,AVphKa...,SRS-ZR...,Sony,Stereo...,wirele...,bhphot...,198.00,198.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
1,AVpgcA...,Sony -...,Sony,Stereo...,speake...,bhphot...,238.00,238.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
2,AVphS0...,Sony -...,Sony,Audio ...,speake...,bhphot...,348.00,348.00,No,0.0,3,1,3,2017,9,March,Wednesday,2017-0...
3,AVpfHc...,SP-FS5...,Pioneer,Speake...,speake...,Bestbu...,129.99,129.99,No,0.0,13,2,3,2017,9,March,Thursday,2017-0...
4,AVphe2...,Refere...,Klipsch,Stereo...,speake...,Bestbu...,549.98,549.98,No,0.0,19,2,3,2017,9,March,Thursday,2017-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6156,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,30,12,2017,52,December,Saturday,2017-1...
6157,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6158,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6159,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...


In [8]:
#Grouping data by Name and Week number.
dfx_mod = dfx.groupby(['Name', 'Week']).agg({'Discount_price':'mean','Imp_count':'sum'}).reset_index()


In [9]:
dfx_mod.sample(5)

,Name,Week,Discount_price,Imp_count
2505,SRS-XB...,18,79.99,5
684,Detail...,43,44.36,10
381,Boyton...,30,64.99,12
479,Chant ...,38,44.99,6
3424,iLive ...,10,33.99,7


#### X-Values = PRICE

In [10]:

#New X dataframe.
x_new = dfx_mod.pivot(index='Week', columns='Name' ,values='Discount_price')
print("Structure of DataFrame = ",x_new.shape)
x_new.sample(5)

Structure of DataFrame =  (42, 223)


Name,151 SE Outdoor Environmental Speakers (White),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",251 Outdoor Environmental Speakers (White),"AW-650 All-Weather Outdoor Speaker (Pair, White)","AW6500 All-Weather Outdoor Speaker (White, Single)",Acoustimass 6 Series V Home Theater Speaker System (Black),"Alpine - 5 x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black""","Alpine - 6 x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black""","Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler,Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units,Alpine PDXM12 1200W Mono RMS Digital Amplifier,"Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set""","Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002""","Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber""","BIC America - 6-1/2 Floor Speaker (Each) - Black""",BOOM 2 Wireless Bluetooth Speaker - Indigo,BRAVEN BRV-HD Wireless Bluetooth Speaker [28 Hour Playtime][Water Resistant] Built-In 8800 mAh Power Bank Charger - Black,"Bic America Rtr1530 15 Rtr Series 3-way Tower Speaker""",Boombox+ Bluetooth Wireless Speaker,Bose SoundLink Color Bluetooth Speaker (Black),Bose® - Soundlink® Color Bluetooth Speaker II - Coral Red,Bose® - Virtually Invisible® 591 In-Ceiling Speakers (Pair) - White,Bose® - Virtually Invisible® 791 In-Ceiling Speakers II (Pair) - White,Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond,Boytone - Portable Bluetooth Speaker - Blue,Braven BRV-BLADE Wireless Portable Bluetooth Speaker [22 Hour Playtime][Waterproof] 4000 mAh Power Bank Charger - Black,Braven Balance Portable Bluetooth Wireless Speaker,Cerwin Vega - SL Series 2-Way Center-Channel Loudspeaker - Black,Cerwin Vega - SL Series 2-Way Center-Channel Speaker - Black,Chant Mini Portable Bluetooth Wireless Speaker (Navy),Chant Sport BT Portable Audio System (Black),Chant Sport BT Portable Audio System (Natural),"Definitive Technology - DT Series 6.5 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - DT Series 8 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - Disappearing Dual 6-1/2 In-Wall Speaker (Each) - Black""",Definitive Technology - Mythos 2-way 175 W Speaker - Pack of 1 - Black,Definitive Technology - ProCinema 600 5.1-Channel Home Theater Speaker System - Black,"Definitive Technology - SuperCube 2000 7-1/2 650W Powered Subwoofer - Black""","Definitive Technology - W9 Dual 5-1/4 180W 2-Way Wireless Speaker (Each) for Streaming Music - Black""",...,Sony - 5.1-Ch. 3D / Smart Blu-ray Home Theater System - Black,"Sony - Core Series 10 115W Active Subwoofer - Black""","Sony - Core Series Dual 5 3-Way Floorstanding Speaker (Each) - Black""",Sony - High Power XB60 Portable Bluetooth Speaker - Black,Sony - XB20 Portable Bluetooth Speaker - Blue,Sony - XB7 Extra Bass Audio System with Bluetooth - Black,Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black,Sony SRSXB3/BLK Portable Wireless Speaker with Bluetooth (Black),Sony Ultra-Portable Bluetooth Speaker,Soundcast - Portable Bluetooth Speaker - Gray/black,"SpeakerCraft - 6-1/2 In-Ceiling Speakers (5-Pack) - White""","SpeakerCraft - Profile CRS6 Two 6-1/2 In-Ceiling Speaker (Each) - Black""",Toshiba - 2.0-Channel Soundbar with 16-Watt Digital Amplifier - Black,Tough Portable Bluetooth Speaker (Black and Silver),Virtually Invisible 891 In-Wall Speakers (Pair),Vizio 38 5.1 Sound Bar System - Black (sb3851-C0),YSP-4300 Digital Sound Projector Wireless Active Subwoofer (Black),YU2 Powered Desktop Speakers (Glossy Red),YU2 Powered Desktop Speakers (Matte Black),YU2 Powered Desktop Speakers (Matte Gray),"Yamaha - 10 250W Powered Subwoofer - High-Gloss Piano Black""","Yamaha - 2.1-Channel Soundbar System with 6-1/2 Wirel

In [11]:
# FOR X VALUES
#Filling missing values with fillna + forward filling
x_new_ = pd.DataFrame(x_new.to_records())
x_new_.fillna(method='ffill', inplace=True)
x_new_.fillna(method='bfill', inplace=True)
x_new_.head(5)

,Week,151 SE Outdoor Environmental Speakers (White),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",251 Outdoor Environmental Speakers (White),"AW-650 All-Weather Outdoor Speaker (Pair, White)","AW6500 All-Weather Outdoor Speaker (White, Single)",Acoustimass 6 Series V Home Theater Speaker System (Black),"Alpine - 5 x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black""","Alpine - 6 x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black""","Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler,Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units,Alpine PDXM12 1200W Mono RMS Digital Amplifier,"Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set""","Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002""","Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber""","BIC America - 6-1/2 Floor Speaker (Each) - Black""",BOOM 2 Wireless Bluetooth Speaker - Indigo,BRAVEN BRV-HD Wireless Bluetooth Speaker [28 Hour Playtime][Water Resistant] Built-In 8800 mAh Power Bank Charger - Black,"Bic America Rtr1530 15 Rtr Series 3-way Tower Speaker""",Boombox+ Bluetooth Wireless Speaker,Bose SoundLink Color Bluetooth Speaker (Black),Bose® - Soundlink® Color Bluetooth Speaker II - Coral Red,Bose® - Virtually Invisible® 591 In-Ceiling Speakers (Pair) - White,Bose® - Virtually Invisible® 791 In-Ceiling Speakers II (Pair) - White,Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond,Boytone - Portable Bluetooth Speaker - Blue,Braven BRV-BLADE Wireless Portable Bluetooth Speaker [22 Hour Playtime][Waterproof] 4000 mAh Power Bank Charger - Black,Braven Balance Portable Bluetooth Wireless Speaker,Cerwin Vega - SL Series 2-Way Center-Channel Loudspeaker - Black,Cerwin Vega - SL Series 2-Way Center-Channel Speaker - Black,Chant Mini Portable Bluetooth Wireless Speaker (Navy),Chant Sport BT Portable Audio System (Black),Chant Sport BT Portable Audio System (Natural),"Definitive Technology - DT Series 6.5 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - DT Series 8 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - Disappearing Dual 6-1/2 In-Wall Speaker (Each) - Black""",Definitive Technology - Mythos 2-way 175 W Speaker - Pack of 1 - Black,Definitive Technology - ProCinema 600 5.1-Channel Home Theater Speaker System - Black,"Definitive Technology - SuperCube 2000 7-1/2 650W Powered Subwoofer - Black""",...,Sony - 5.1-Ch. 3D / Smart Blu-ray Home Theater System - Black,"Sony - Core Series 10 115W Active Subwoofer - Black""","Sony - Core Series Dual 5 3-Way Floorstanding Speaker (Each) - Black""",Sony - High Power XB60 Portable Bluetooth Speaker - Black,Sony - XB20 Portable Bluetooth Speaker - Blue,Sony - XB7 Extra Bass Audio System with Bluetooth - Black,Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black,Sony SRSXB3/BLK Portable Wireless Speaker with Bluetooth (Black),Sony Ultra-Portable Bluetooth Speaker,Soundcast - Portable Bluetooth Speaker - Gray/black,"SpeakerCraft - 6-1/2 In-Ceiling Speakers (5-Pack) - White""","SpeakerCraft - Profile CRS6 Two 6-1/2 In-Ceiling Speaker (Each) - Black""",Toshiba - 2.0-Channel Soundbar with 16-Watt Digital Amplifier - Black,Tough Portable Bluetooth Speaker (Black and Silver),Virtually Invisible 891 In-Wall Speakers (Pair),Vizio 38 5.1 Sound Bar System - Black (sb3851-C0),YSP-4300 Digital Sound Projector Wireless Active Subwoofer (Black),YU2 Powered Desktop Speakers (Glossy Red),YU2 Powered Desktop Speakers (Matte Black),YU2 Powered Desktop Speakers (Matte Gray),"Yamaha - 10 250W Powered Subwoofer - High-Gloss Piano Black""","Yamaha - 2.1-Channel Soundbar System with 6-1/2 Wireless Subwoofer and 200-Watt Digital Amplifier - Black""","Yamaha - 6-1/2 130-Watt Powered Subwoofer - Bla

#### Y-Values = DEMAND or IMPRESSIONS

In [12]:
#New Y dataframe.
y_new = dfx_mod.pivot(index='Week', columns='Name', values='Imp_count')
print("Structure of DataFrame = ",y_new.shape)
y_new.sample(5)

Structure of DataFrame =  (42, 223)


Name,151 SE Outdoor Environmental Speakers (White),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",251 Outdoor Environmental Speakers (White),"AW-650 All-Weather Outdoor Speaker (Pair, White)","AW6500 All-Weather Outdoor Speaker (White, Single)",Acoustimass 6 Series V Home Theater Speaker System (Black),"Alpine - 5 x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black""","Alpine - 6 x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black""","Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler,Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units,Alpine PDXM12 1200W Mono RMS Digital Amplifier,"Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set""","Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002""","Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber""","BIC America - 6-1/2 Floor Speaker (Each) - Black""",BOOM 2 Wireless Bluetooth Speaker - Indigo,BRAVEN BRV-HD Wireless Bluetooth Speaker [28 Hour Playtime][Water Resistant] Built-In 8800 mAh Power Bank Charger - Black,"Bic America Rtr1530 15 Rtr Series 3-way Tower Speaker""",Boombox+ Bluetooth Wireless Speaker,Bose SoundLink Color Bluetooth Speaker (Black),Bose® - Soundlink® Color Bluetooth Speaker II - Coral Red,Bose® - Virtually Invisible® 591 In-Ceiling Speakers (Pair) - White,Bose® - Virtually Invisible® 791 In-Ceiling Speakers II (Pair) - White,Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond,Boytone - Portable Bluetooth Speaker - Blue,Braven BRV-BLADE Wireless Portable Bluetooth Speaker [22 Hour Playtime][Waterproof] 4000 mAh Power Bank Charger - Black,Braven Balance Portable Bluetooth Wireless Speaker,Cerwin Vega - SL Series 2-Way Center-Channel Loudspeaker - Black,Cerwin Vega - SL Series 2-Way Center-Channel Speaker - Black,Chant Mini Portable Bluetooth Wireless Speaker (Navy),Chant Sport BT Portable Audio System (Black),Chant Sport BT Portable Audio System (Natural),"Definitive Technology - DT Series 6.5 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - DT Series 8 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - Disappearing Dual 6-1/2 In-Wall Speaker (Each) - Black""",Definitive Technology - Mythos 2-way 175 W Speaker - Pack of 1 - Black,Definitive Technology - ProCinema 600 5.1-Channel Home Theater Speaker System - Black,"Definitive Technology - SuperCube 2000 7-1/2 650W Powered Subwoofer - Black""","Definitive Technology - W9 Dual 5-1/4 180W 2-Way Wireless Speaker (Each) for Streaming Music - Black""",...,Sony - 5.1-Ch. 3D / Smart Blu-ray Home Theater System - Black,"Sony - Core Series 10 115W Active Subwoofer - Black""","Sony - Core Series Dual 5 3-Way Floorstanding Speaker (Each) - Black""",Sony - High Power XB60 Portable Bluetooth Speaker - Black,Sony - XB20 Portable Bluetooth Speaker - Blue,Sony - XB7 Extra Bass Audio System with Bluetooth - Black,Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black,Sony SRSXB3/BLK Portable Wireless Speaker with Bluetooth (Black),Sony Ultra-Portable Bluetooth Speaker,Soundcast - Portable Bluetooth Speaker - Gray/black,"SpeakerCraft - 6-1/2 In-Ceiling Speakers (5-Pack) - White""","SpeakerCraft - Profile CRS6 Two 6-1/2 In-Ceiling Speaker (Each) - Black""",Toshiba - 2.0-Channel Soundbar with 16-Watt Digital Amplifier - Black,Tough Portable Bluetooth Speaker (Black and Silver),Virtually Invisible 891 In-Wall Speakers (Pair),Vizio 38 5.1 Sound Bar System - Black (sb3851-C0),YSP-4300 Digital Sound Projector Wireless Active Subwoofer (Black),YU2 Powered Desktop Speakers (Glossy Red),YU2 Powered Desktop Speakers (Matte Black),YU2 Powered Desktop Speakers (Matte Gray),"Yamaha - 10 250W Powered Subwoofer - High-Gloss Piano Black""","Yamaha - 2.1-Channel Soundbar System with 6-1/2 Wirel

In [13]:
# FOR Y VALUES
#Filling missing values with fillna + forward filling
y_new_ = pd.DataFrame(y_new.to_records())
y_new_.fillna(method='ffill', inplace=True)
y_new_.fillna(method='bfill', inplace=True)
y_new_.head(5)

,Week,151 SE Outdoor Environmental Speakers (White),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",251 Outdoor Environmental Speakers (White),"AW-650 All-Weather Outdoor Speaker (Pair, White)","AW6500 All-Weather Outdoor Speaker (White, Single)",Acoustimass 6 Series V Home Theater Speaker System (Black),"Alpine - 5 x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black""","Alpine - 6 x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black""","Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler,Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units,Alpine PDXM12 1200W Mono RMS Digital Amplifier,"Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set""","Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002""","Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber""","BIC America - 6-1/2 Floor Speaker (Each) - Black""",BOOM 2 Wireless Bluetooth Speaker - Indigo,BRAVEN BRV-HD Wireless Bluetooth Speaker [28 Hour Playtime][Water Resistant] Built-In 8800 mAh Power Bank Charger - Black,"Bic America Rtr1530 15 Rtr Series 3-way Tower Speaker""",Boombox+ Bluetooth Wireless Speaker,Bose SoundLink Color Bluetooth Speaker (Black),Bose® - Soundlink® Color Bluetooth Speaker II - Coral Red,Bose® - Virtually Invisible® 591 In-Ceiling Speakers (Pair) - White,Bose® - Virtually Invisible® 791 In-Ceiling Speakers II (Pair) - White,Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond,Boytone - Portable Bluetooth Speaker - Blue,Braven BRV-BLADE Wireless Portable Bluetooth Speaker [22 Hour Playtime][Waterproof] 4000 mAh Power Bank Charger - Black,Braven Balance Portable Bluetooth Wireless Speaker,Cerwin Vega - SL Series 2-Way Center-Channel Loudspeaker - Black,Cerwin Vega - SL Series 2-Way Center-Channel Speaker - Black,Chant Mini Portable Bluetooth Wireless Speaker (Navy),Chant Sport BT Portable Audio System (Black),Chant Sport BT Portable Audio System (Natural),"Definitive Technology - DT Series 6.5 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - DT Series 8 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - Disappearing Dual 6-1/2 In-Wall Speaker (Each) - Black""",Definitive Technology - Mythos 2-way 175 W Speaker - Pack of 1 - Black,Definitive Technology - ProCinema 600 5.1-Channel Home Theater Speaker System - Black,"Definitive Technology - SuperCube 2000 7-1/2 650W Powered Subwoofer - Black""",...,Sony - 5.1-Ch. 3D / Smart Blu-ray Home Theater System - Black,"Sony - Core Series 10 115W Active Subwoofer - Black""","Sony - Core Series Dual 5 3-Way Floorstanding Speaker (Each) - Black""",Sony - High Power XB60 Portable Bluetooth Speaker - Black,Sony - XB20 Portable Bluetooth Speaker - Blue,Sony - XB7 Extra Bass Audio System with Bluetooth - Black,Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black,Sony SRSXB3/BLK Portable Wireless Speaker with Bluetooth (Black),Sony Ultra-Portable Bluetooth Speaker,Soundcast - Portable Bluetooth Speaker - Gray/black,"SpeakerCraft - 6-1/2 In-Ceiling Speakers (5-Pack) - White""","SpeakerCraft - Profile CRS6 Two 6-1/2 In-Ceiling Speaker (Each) - Black""",Toshiba - 2.0-Channel Soundbar with 16-Watt Digital Amplifier - Black,Tough Portable Bluetooth Speaker (Black and Silver),Virtually Invisible 891 In-Wall Speakers (Pair),Vizio 38 5.1 Sound Bar System - Black (sb3851-C0),YSP-4300 Digital Sound Projector Wireless Active Subwoofer (Black),YU2 Powered Desktop Speakers (Glossy Red),YU2 Powered Desktop Speakers (Matte Black),YU2 Powered Desktop Speakers (Matte Gray),"Yamaha - 10 250W Powered Subwoofer - High-Gloss Piano Black""","Yamaha - 2.1-Channel Soundbar System with 6-1/2 Wireless Subwoofer and 200-Watt Digital Amplifier - Black""","Yamaha - 6-1/2 130-Watt Powered Subwoofer - Bla

In [48]:
x_new_["BOOM 2 Wireless Bluetooth Speaker - Indigo"].value_counts()

199.99    27
149.99    11
179.99     2
129.99     1
147.02     1
Name: BOOM 2 Wireless Bluetooth Speaker - Indigo, dtype: int64

### Trial on single product.

In [52]:
temp_df = pd.DataFrame()
temp_df['x'] = x_new_["BOOM 2 Wireless Bluetooth Speaker - Indigo"]
temp_df['y'] = y_new_["BOOM 2 Wireless Bluetooth Speaker - Indigo"]
x_value = temp_df[['x']]
y_value = temp_df['y']
#Creating Linear Model
model = LinearRegression()
model.fit(x_value, y_value)

#Eval
slope = model.coef_ 
intercept = model.intercept_

##
y_pred = model.predict(x_value)
r_squared = r2_score(y_value, y_pred)

print("#"*10)
print("Slope: ", slope)
print("Intercept: ", intercept)
print("R-squared: ", r_squared*100)
print("#"*10)

model.predict([[186]])
print("If price of BOOM 2 Wireless Bluetooth Speaker - Indigo is set to 200$, demand will be ",model.predict([[149.99]]))



##########
Slope:  [0.1641456]
Intercept:  -16.945789880176054
R-squared:  47.368571594798894
##########
If price of BOOM 2 Wireless Bluetooth Speaker - Indigo is set to 200$, demand will be  [7.67440797]


In [27]:
print("Slope: ", slope)
print("Intercept: ", intercept)
print("R-squared: ", r_squared)

Slope:  [0.1641456]
Intercept:  -16.945789880176054
R-squared:  0.4736857159479889


Price elasticity = Change in Quanity sold / Change in the Price

In [16]:
for col in x_new_.columns[1:]:
    print(col)
    

151 SE Outdoor Environmental Speakers (White)
2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693"
251 Outdoor Environmental Speakers (White)
AW-650 All-Weather Outdoor Speaker (Pair, White)
AW6500 All-Weather Outdoor Speaker (White, Single)
Acoustimass 6 Series V Home Theater Speaker System (Black)
Alpine - 5 x 7" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black"
Alpine - 6 x 9" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black"
Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black"
Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler
Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units
Alpine PDXM12 1200W Mono RMS Digital Amplifier
Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set"
Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002"
Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber"
BIC America - 6-1/2 Floor Spea

In [55]:
def multizen(cox):
    temp_df = pd.DataFrame()
    temp_df['x'] = x_new_[cox]
    temp_df['y'] = y_new_[cox]
    x_value = temp_df[['x']]
    y_value = temp_df['y']
    #Creating Linear Model
    model = LinearRegression()
    model.fit(x_value, y_value)

    #Eval
    slope = model.coef_ 
    intercept = model.intercept_

    #####
    y_pred = model.predict(x_value)
    r_squared = r2_score(y_value, y_pred)

    print("#"*10)
    print("Slope: ", slope)
    print("Intercept: ", intercept)
    print("R-squared: ", r_squared*100,"%")
    print("#"*10)


In [56]:
for col in x_new_.columns[1:]:
    multizen(col)
    

##########
Slope:  [-7.29517396]
Intercept:  2047.7805836138975
R-squared:  61.21676134024862 %
##########
##########
Slope:  [-0.02944041]
Intercept:  7.8094831743868305
R-squared:  2.8458302895660648 %
##########
##########
Slope:  [-7.01754386]
Intercept:  2814.3609022556266
R-squared:  54.37073155423422 %
##########
##########
Slope:  [0.04252188]
Intercept:  -12.21367248665909
R-squared:  46.71228801401854 %
##########
##########
Slope:  [0.21661525]
Intercept:  -20.088119238352032
R-squared:  13.943268689915167 %
##########
##########
Slope:  [0.04337042]
Intercept:  -16.98016444498456
R-squared:  17.66463462422283 %
##########
##########
Slope:  [1.10706599]
Intercept:  -67.60044922146604
R-squared:  79.07439098004663 %
##########
##########
Slope:  [0.16481393]
Intercept:  -5.7096108815616935
R-squared:  16.71439657695436 %
##########
##########
Slope:  [0.86794993]
Intercept:  -66.16769551719082
R-squared:  66.53767193478414 %
##########
##########
Slope:  [0.18504111]
Interce

#### Forward Selection Method

In [17]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=3,direction="forward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [0 2 3]


#### Backward Selection Method

In [18]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=2,direction="backward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [2 3]


#### Based on P Value

In [19]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import f_regression

# load the Iris dataset
iris = load_iris()

# extract the features and target variable
X = iris.data
y = iris.target

# calculate the F-test statistic and p-value for each feature
f_test, p_values = f_regression(X, y)

# print the p-value of each feature
for i, p_value in enumerate(p_values):
    print(f"Feature {i+1}: p-value = {p_value:.4f}")


# if p < 0.05

Feature 1: p-value = 0.0000
Feature 2: p-value = 0.0000
Feature 3: p-value = 0.0000
Feature 4: p-value = 0.0000
